# Lecture et traitement de données

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("donnees_voitures/donnees.csv", sep=",", header=None)

# Nom des colonnes :
df.columns = [
    "Prix",
    "Vitesse_Max",
    "Conso_moy",
    "Dis_Freinage",
    "Confort",
    "Vol_Coffre",
    "Acceleration"
]

# Ajouter les noms des alternatives
model_names = [
    "Alfa_156",
    "Audi_A4",
    "Cit_Xantia",
    "Peugeot_406",
    "Saab_TID",
    "Rnlt_Laguna",
    "VW_Passat",
    "BMW_320d",
    "Cit_Xsara",
    "Rnlt_Safrane"
]
df["Modele"] = model_names

# Debug
print("Données chargées :")
print(df)

# Liste de critères et orientation --
criteria = [
    "Prix",
    "Vitesse_Max",
    "Conso_moy",
    "Dis_Freinage",
    "Confort",
    "Vol_Coffre",
    "Acceleration"
]
# True => critère à maximiser, False => critère à minimiser
maximize = {
    "Prix": False,
    "Vitesse_Max": True,
    "Conso_moy": False,
    "Dis_Freinage": False,
    "Confort": False,
    "Vol_Coffre": True,
    "Acceleration": False
}

#    Si un critère est à minimiser => on multiplie par -1
for c in criteria:
    if not maximize[c]:
        df[c] = -df[c]

# Debug
print("\nAprès conversion en 'max', puis normalisation :")
print(df)

# Récupérer la liste des alternatives 
alt_list = df["Modele"].tolist()

Données chargées :
    Prix  Vitesse_Max  Conso_moy  Dis_Freinage  Confort  Vol_Coffre  \
0  23817          201        8.0          39.6        6         378   
1  25771          195        5.7          35.8        7         440   
2  25496          195        7.9          37.0        2         480   
3  25649          191        8.3          34.4        2         430   
4  26183          199        7.8          35.7        5         494   
5  23664          194        7.7          37.4        4         452   
6  23344          195        7.6          34.4        3         475   
7  26260          209        6.6          36.6        4         440   
8  19084          182        6.4          40.6        8         408   
9  29160          203        7.5          34.5        1         520   

   Acceleration        Modele  
0          31.2      Alfa_156  
1          33.0       Audi_A4  
2          34.0    Cit_Xantia  
3          34.6   Peugeot_406  
4          32.0      Saab_TID  
5      

# Les Poids et paramètres de préférences

In [2]:
# Pondérations : La somme est 1
weights = {
    "Prix": 0.2,
    "Vitesse_Max": 0.15,
    "Conso_moy": 0.15,
    "Dis_Freinage": 0.15,
    "Confort": 0.15,
    "Vol_Coffre": 0.10,
    "Acceleration": 0.10
}

# Seuils
p_k = {
    "Prix": 1600,
    "Vitesse_Max": 8,
    "Conso_moy": 1,
    "Dis_Freinage": 1,
    "Confort": 1,
    "Vol_Coffre": 30,
    "Acceleration": 1
}



# On fixe le seuil de concordance
s = 0.75

# Veto par critère (pour la partie discordance)
veto = {
    "Prix": 2500,
    "Vitesse_Max": 10,
    "Conso_moy": 1,
    "Dis_Freinage": 2,
    "Confort": 1,
    "Vol_Coffre": 40,
    "Acceleration": 1
    }


# Promethee I & II

In [3]:
# Fonction de préférence promethee
def preference_promethee(d, p=0.0):
    
    if p == 0:
        # Fonction usuelle
        return 1.0 if d > 0 else 0.0
    else:
        # Fonction linéaire
        if d <= 0:
            return 0.0
        elif d >= p:
            return 1.0
        else:
            return d / p

## Promethee Sans P_k

In [4]:
# Construction de la matrice pi(a,b) = somme pondérée

pi_matrix = {}
for a in alt_list:
    pi_matrix[a] = {}
    row_a = df.loc[df["Modele"] == a]
    
    for b in alt_list:
        if a == b:
            # Diagonale : on peut mettre 0 ou '-'
            pi_matrix[a][b] = 0.0
            continue
        
        row_b = df.loc[df["Modele"] == b]
        pi_ab = 0.0
        for c in criteria:
            val_a = row_a[c].values[0]
            val_b = row_b[c].values[0]
            d = val_a - val_b
            
            pref = preference_promethee(d, p=0)
            pi_ab += weights[c] * pref
        
        pi_matrix[a][b] = pi_ab

df_pi = pd.DataFrame(pi_matrix).T 


phi_plus_series = df_pi.sum(axis=1)
phi_minus_series = df_pi.sum(axis=0)

# PROMETHEE II : calcul du flux net

phi_net_series = phi_plus_series - phi_minus_series

df_pi["\u03C6+"] = phi_plus_series
df_pi["\u03C6_net"] = phi_net_series

df_pi.loc["\u03C6-"] = list(phi_minus_series) + [phi_plus_series.sum(), phi_net_series.sum()]

print("\n=== Promethee sans P_k ===")
print("\n=== Matrice pi(a,b) + flux PROMETHEE ===")
print(df_pi)

# On classe les alternatives selon le flux net, ordre décroissant
prom_ii_ranking = sorted(phi_net_series.items(), key=lambda x: x[1], reverse=True)

print("\n=== PROMETHEE II : Classement final (flux net) ===")
for rank, (alt, val) in enumerate(prom_ii_ranking, start=1):
    print(f"Rang {rank} : {alt} (phi_net = {val:.3f})")


=== Promethee sans P_k ===

=== Matrice pi(a,b) + flux PROMETHEE ===
              Alfa_156  Audi_A4  Cit_Xantia  Peugeot_406  Saab_TID  \
Alfa_156          0.00     0.60        0.45         0.60      0.45   
Audi_A4           0.40     0.00        0.40         0.50      0.35   
Cit_Xantia        0.55     0.45        0.00         0.70      0.35   
Peugeot_406       0.40     0.50        0.15         0.00      0.50   
Saab_TID          0.55     0.65        0.65         0.50      0.00   
Rnlt_Laguna       0.75     0.45        0.45         0.70      0.50   
VW_Passat         0.75     0.60        0.60         0.70      0.65   
BMW_320d          0.80     0.40        0.55         0.50      0.55   
Cit_Xsara         0.45     0.20        0.45         0.45      0.35   
Rnlt_Safrane      0.70     0.65        0.80         0.65      0.70   
φ-                5.35     4.50        4.50         5.30      4.40   

              Rnlt_Laguna  VW_Passat  BMW_320d  Cit_Xsara  Rnlt_Safrane  \
Alfa_156      

# Promethee avec P_k

In [5]:

pi_matrix = {}
for a in alt_list:
    pi_matrix[a] = {}
    row_a = df.loc[df["Modele"] == a]
    
    for b in alt_list:
        if a == b:
            # Diagonale : on peut mettre 0 ou '-'
            pi_matrix[a][b] = 0.0
            continue
        
        row_b = df.loc[df["Modele"] == b]
        pi_ab = 0.0
        for c in criteria:
            val_a = row_a[c].values[0]
            val_b = row_b[c].values[0]
            d = val_a - val_b
            
            # Utilisation du seuil p_k[c] pour la fonction linéaire
            pref = preference_promethee(d, p_k[c])
            pi_ab += weights[c] * pref
        
        pi_matrix[a][b] = pi_ab

df_pi = pd.DataFrame(pi_matrix).T

phi_plus_series = df_pi.sum(axis=1)
phi_minus_series = df_pi.sum(axis=0)
phi_net_series = phi_plus_series - phi_minus_series

df_pi["\u03C6+"] = phi_plus_series
df_pi["\u03C6_net"] = phi_net_series
df_pi.loc["\u03C6-"] = list(phi_minus_series) + [phi_plus_series.sum(), phi_net_series.sum()]

print("\n=== Promethee avec P_k ===")
print("\n=== Matrice pi(a,b) + flux PROMETHEE ===")
print(df_pi)

# On classe les alternatives selon le flux net, ordre décroissant
prom_ii_ranking = sorted(phi_net_series.items(), key=lambda x: x[1], reverse=True)

print("\n=== PROMETHEE II : Classement final (flux net) ===")
for rank, (alt, val) in enumerate(prom_ii_ranking, start=1):
    print(f"Rang {rank} : {alt} (phi_net = {val:.3f})")


=== Promethee avec P_k ===

=== Matrice pi(a,b) + flux PROMETHEE ===
              Alfa_156   Audi_A4  Cit_Xantia  Peugeot_406  Saab_TID  \
Alfa_156      0.000000  0.562500    0.412500     0.495000  0.317500   
Audi_A4       0.400000  0.000000    0.400000     0.358333  0.201500   
Cit_Xantia    0.415000  0.284375    0.000000     0.314125  0.235875   
Peugeot_406   0.400000  0.315250    0.150000     0.000000  0.366750   
Saab_TID      0.430000  0.440000    0.386667     0.425000  0.000000   
Rnlt_Laguna   0.464125  0.390000    0.250000     0.499583  0.365000   
VW_Passat     0.519125  0.600000    0.435000     0.580000  0.530000   
BMW_320d      0.730000  0.400000    0.460000     0.433333  0.550000   
Cit_Xsara     0.450000  0.200000    0.400000     0.450000  0.350000   
Rnlt_Safrane  0.512500  0.650000    0.710000     0.620000  0.506667   
φ-            4.320750  3.842125    3.604167     4.175375  3.423292   

              Rnlt_Laguna  VW_Passat  BMW_320d  Cit_Xsara  Rnlt_Safrane  \
Al

# ELECTRE

In [6]:
def local_concordance(val_a, val_b, p):
    """
    Si p > 0 : fonction linéaire (IS)
        - d >= 0       -> 1.0
        - d <= -p      -> 0.0
        - sinon        -> 1 + d/p
    Sinon : fonction binaire (IV)
        - 1 si val_a >= val_b, 0 sinon
    """
    d = val_a - val_b
    if p > 0:
        if d >= 0:
            return 1.0
        elif d <= -p:
            return 0.0
        else:
            return 1.0 + d / p
    else:
        return 1.0 if val_a >= val_b else 0.0
    


def check_nondiscordance(a, b, df, criteria, veto):
    row_a = df.loc[df["Modele"] == a]
    row_b = df.loc[df["Modele"] == b]
    for c in criteria:
        if (row_b[c].values[0] - row_a[c].values[0]) >= veto[c]:
            return 0
    return 1

## ELECTRE IV

In [7]:
def global_concordance_iv(a, b, df, criteria, weights):
    row_a = df.loc[df["Modele"] == a]
    row_b = df.loc[df["Modele"] == b]
    sum_w = sum(weights[c] for c in criteria)
    sum_c = 0.0
    for c in criteria:
        va = row_a[c].values[0]
        vb = row_b[c].values[0]
        c_k = local_concordance(va, vb, p=0.0)
        sum_c += weights[c] * c_k
    return sum_c / sum_w

# Construction de M1 : matrice de concordance
M1_concordance = {}
for a in alt_list:
    M1_concordance[a] = {}
    for b in alt_list:
        if a == b:
            M1_concordance[a][b] = 0
            continue
        c_ab = global_concordance_iv(a, b, df, criteria, weights)
        
        M1_concordance[a][b] = c_ab

# Construction de M2 : matrice de non-discordance binaire
M2_nondisco = {}
for a in alt_list:
    M2_nondisco[a] = {}
    for b in alt_list:
        if a == b:
            M2_nondisco[a][b] = 0
            continue
        M2_nondisco[a][b] = check_nondiscordance(a, b, df, criteria, veto)


# On dit que a surclasse b si : M1(a,b)>=1 ET M2(a,b)=1
surclassement = {}
for a in alt_list:
    surclassement[a] = {}
    for b in alt_list:
        if a == b:
            surclassement[a][b] = 0
            continue

        if (M1_concordance[a][b] >= s) and (M2_nondisco[a][b] == 1):
            surclassement[a][b] = 1
        else:
            surclassement[a][b] = 0

df_M1 = pd.DataFrame(M1_concordance).T 
df_M2 = pd.DataFrame(M2_nondisco).T
df_surcl = pd.DataFrame(surclassement).T

print("\n=== M1 : Concordance >= 0.75 ===")
print(df_M1)

print("\n=== M2 : Non-Discordance (0/1) ===")
print(df_M2)

print("\n=== Surclassement final (aSb ?) ===")
print(df_surcl)

electre_score = {}
for a in alt_list:
    score_plus = sum(surclassement[a][b] for b in alt_list if b != a)
    score_minus = sum(surclassement[b][a] for b in alt_list if b != a)
    electre_score[a] = score_plus - score_minus

ranking = sorted(electre_score.items(), key=lambda x: x[1], reverse=True)

print("\n=== Classement final ===")
for rank, (alt, sc) in enumerate(ranking, start=1):
    print(f"Rang {rank} -> {alt}, score = {sc}")


=== M1 : Concordance >= 0.75 ===
              Alfa_156  Audi_A4  Cit_Xantia  Peugeot_406  Saab_TID  \
Alfa_156          0.00     0.60        0.45         0.60      0.45   
Audi_A4           0.40     0.00        0.55         0.50      0.35   
Cit_Xantia        0.55     0.60        0.00         0.85      0.35   
Peugeot_406       0.40     0.50        0.30         0.00      0.50   
Saab_TID          0.55     0.65        0.65         0.50      0.00   
Rnlt_Laguna       0.75     0.45        0.45         0.70      0.50   
VW_Passat         0.75     0.75        0.75         0.85      0.65   
BMW_320d          0.80     0.50        0.55         0.50      0.55   
Cit_Xsara         0.45     0.20        0.45         0.45      0.35   
Rnlt_Safrane      0.70     0.65        0.80         0.65      0.80   

              Rnlt_Laguna  VW_Passat  BMW_320d  Cit_Xsara  Rnlt_Safrane  
Alfa_156             0.25       0.25      0.20       0.55          0.30  
Audi_A4              0.55       0.40      0.60 

# ELECTRE IS

In [8]:
def global_concordance_is(a, b):
    row_a = df.loc[df["Modele"] == a]
    row_b = df.loc[df["Modele"] == b]

    sum_c = 0.0
    for c in criteria:
        va = row_a[c].values[0]
        vb = row_b[c].values[0]
        p_val = p_k[c]
        c_k = local_concordance(va, vb, p_val)
        sum_c += weights[c] * c_k
    return sum_c


M1_concordance = {}
M2_nondisco = {}

for a in alt_list:
    M1_concordance[a] = {}
    M2_nondisco[a] = {}
    for b in alt_list:
        if a == b:
            M1_concordance[a][b] = 0.0
            M2_nondisco[a][b] = 0
            continue

        c_ab = global_concordance_is(a, b)
        M1_concordance[a][b] = c_ab

        nd_ab = check_nondiscordance(a, b,df, criteria, veto)
        M2_nondisco[a][b] = nd_ab

surclassement = {}
for a in alt_list:
    surclassement[a] = {}
    for b in alt_list:
        if a == b:
            surclassement[a][b] = 0
            continue

        if (M1_concordance[a][b] >= s) and (M2_nondisco[a][b] == 1):
            surclassement[a][b] = 1
        else:
            surclassement[a][b] = 0


print("\n=== ELECTRE IS: Table de concordance (c(a,b)) ===")
df_concordance = pd.DataFrame(M1_concordance).T 
print(df_concordance)

print("\n=== ELECTRE IS: Table de non-discordance (0/1) ===")
df_nondisco = pd.DataFrame(M2_nondisco).T
print(df_nondisco)

print(f"\n=== Surclassement final (seuil c={s}) ===")
df_surclassement = pd.DataFrame(surclassement).T
print(df_surclassement)

print("\n=== Détail des alternatives surclassées par chaque option ===")
for a in alt_list:
    outranked = [b for b in alt_list if surclassement[a][b] == 1]
    print(f"{a} surclasse: {outranked}")

electre_score = {}
for a in alt_list:
    score_plus = sum(surclassement[a][b] for b in alt_list if b != a)
    score_minus = sum(surclassement[b][a] for b in alt_list if b != a)
    score_net = score_plus - score_minus
    electre_score[a] = score_net

ranking = sorted(electre_score.items(), key=lambda x: x[1], reverse=True)

print("\n=== CLASSEMENT FINAL (Score net) ===")
for rank, (alt, sc) in enumerate(ranking, start=1):
    print(f"Rang {rank}: {alt} (score_net = {sc})")



=== ELECTRE IS: Table de concordance (c(a,b)) ===
              Alfa_156   Audi_A4  Cit_Xantia  Peugeot_406  Saab_TID  \
Alfa_156       0.00000  0.600000    0.585000     0.600000  0.570000   
Audi_A4        0.43750  0.000000    0.715625     0.684750  0.560000   
Cit_Xantia     0.58750  0.600000    0.000000     0.850000  0.613333   
Peugeot_406    0.50500  0.641667    0.685875     0.000000  0.575000   
Saab_TID       0.68250  0.798500    0.764125     0.633250  0.000000   
Rnlt_Laguna    0.76875  0.601250    0.677917     0.700000  0.556250   
VW_Passat      0.78750  0.790000    0.833333     0.850000  0.761667   
BMW_320d       0.80000  0.683875    0.654500     0.623625  0.755375   
Cit_Xsara      0.45000  0.295000    0.450000     0.476667  0.350000   
Rnlt_Safrane   0.72000  0.650000    0.800000     0.785000  0.800000   

              Rnlt_Laguna  VW_Passat  BMW_320d  Cit_Xsara  Rnlt_Safrane  
Alfa_156         0.535875   0.480875  0.270000       0.55      0.487500  
Audi_A4          0.